In [1]:
"""
    Imports
"""

import requests
from io import BytesIO
from PIL import Image
import os
from itertools import islice

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets

import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as T
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchmetrics.classification import BinaryF1Score

from torchvision.models import resnet152
from torchvision.models.resnet import ResNet152_Weights

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

from utils import *

In [2]:
"""
    https://towardsdatascience.com/a-friendly-introduction-to-siamese-networks-85ab17522942
"""

class SiameseNetworkClassifier(nn.Module):
    def __init__(self, device='mps'):
        super(SiameseNetworkClassifier, self).__init__()

        # Replace with frozen ResNet152 feature layers
        resnet = resnet152(weights=ResNet152_Weights.IMAGENET1K_V1)
        # Freeze all layers
        for param in resnet.parameters():
            param.requires_grad = False

        # Split the model into frozen and hot layers
        self.frozen = nn.Sequential(*list(resnet.children())[:-1]) 
        self.hot = nn.Sequential(
            nn.Flatten(),
            nn.Linear(resnet.fc.in_features, 50)
        )

        self.threshold = torch.tensor(0.)

        self.device = torch.device(device)
        self.to(self.device)

    def forward(self, images1, images2):
        output1 = self.hot(self.frozen(images1))
        output2 = self.hot(self.frozen(images2))
        return F.pairwise_distance(output1, output2)
        
    def update_threshold(self, loader, max_batches=None):
        self.eval()
        with torch.no_grad():
            distances = []
            labels = []
            for images1, images2, equals in islice(loader, max_batches):
                distance = self.forward(images1.to(self.device), images2.to(self.device))
                distances.append(distance.cpu())
                labels.append(equals)
    
            distances = torch.cat(distances)
            labels = torch.cat(labels)
            log_reg = LogisticRegression(penalty=None)
            log_reg.fit(distances.reshape((-1, 1)), labels)
            self.threshold = (-log_reg.intercept_ / log_reg.coef_).item()

    # TODO refactor this method so we don't have to call .to(self.device) ? 
    def predict(self, images1, images2):
        self.eval()
        with torch.no_grad():
            images1 = images1.to(self.device)
            images2 = images2.to(self.device)
            distances = self.forward(images1, images2)
            return (distances < self.threshold).int().cpu()

In [3]:
batch_size = 32
max_submit_id = 22661

In [8]:
transform = T.Compose([
    T.Resize(300),
    T.CenterCrop(300),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

df_slices = list(split_dataframe(pd.read_csv('data/train.csv'), (0.9, 0.01, 0.04, 0.05)))
datasets = [ImageDataset(df, transform=transform) for df in df_slices]
loaders = [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in datasets]

train_data, train_threshold_data, valid_data, test_data = df_slices
train_dataset, train_threshold_dataset, valid_dataset, test_dataset = datasets
train_loader, train_threshold_loader, valid_loader, test_loader = loaders

submit_df = pd.read_csv('data/submit.csv')
submit_dataset = ImageDataset(submit_df, transform=transform)
submit_loader = DataLoader(submit_dataset, batch_size=batch_size, shuffle=True)


In [5]:
from model_specific import train, evaluate

model = SiameseNetworkClassifier()
train(model, *loaders, epochs=3, max_batches=30, print_fscore=True)

Epoch 0:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0 | Loss:0.15543749928474426


Evaluating model:   0%|          | 0/30 [00:00<?, ?it/s]

Train fscore: 0.9988178014755249


Evaluating model:   0%|          | 0/30 [00:00<?, ?it/s]

...

Train fscore: 0.997597336769104


Evaluating model:   0%|          | 0/30 [00:00<?, ?it/s]

Valid fscore: 0.9962907433509827


Evaluating model:   0%|          | 0/30 [00:00<?, ?it/s]

Test fscore: 0.9964053630828857


In [6]:
"""
    Check on what's wrong with our model
"""

mislabeled(model, valid_loader)

Button(description='Next Images', style=ButtonStyle())

Output()

In [10]:
"""
    Save predictions for the submission
"""

max_submit_id = 22661
save_submission(model, submit_loader, max_submit_id)